
WEEK 8 – Diffie-Hellman Key Exchange

In [ ]:
# Diffie-Hellman Key Exchange
def power_mod(base, exponent, modulus):
    return pow(base, exponent, modulus)

p = 23  # prime number
g = 5   # generator

a = 6   # Alice's private key
b = 15  # Bob's private key

A = power_mod(g, a, p)  # Alice's public key
B = power_mod(g, b, p)  # Bob's public key

shared_secret_alice = power_mod(B, a, p)
shared_secret_bob = power_mod(A, b, p)

print("Shared Secret (Alice):", shared_secret_alice)
print("Shared Secret (Bob):", shared_secret_bob)

**WEEK 9 – SHA-256 Hash Function**


In [ ]:
import hashlib

def sha256_hex(msg: bytes) -> str:
    h = hashlib.sha256()
    h.update(msg)
    return h.hexdigest()

messages = [b"", b"abc", b"Hello abc", b"Hai abc"]

for m in messages:
    print(f"Message: {m}")
    print(f"SHA256: {sha256_hex(m)}\n")

WEEK 10 – Digital Signature Standard (DSS/DSA)

In [ ]:
from cryptography.hazmat.primitives.asymmetric import dsa
from cryptography.hazmat.primitives import hashes

def generate_keys():
    private_key = dsa.generate_private_key(key_size=1024)
    public_key = private_key.public_key()
    return private_key, public_key

def sign_message(private_key, message):
    return private_key.sign(message, hashes.SHA256())

def verify_signature(public_key, message, signature):
    try:
        public_key.verify(signature, message, hashes.SHA256())
        return True
    except Exception:
        return False

message = b"Hello, this message will be signed."

priv, pub = generate_keys()
signature = sign_message(priv, message)

print("Signature (Hex):", signature.hex())
print("Verification:", verify_signature(pub, message, signature))

WEEK 11 – SSL and Intrusion Detection System (IDS)

Server Code

In [ ]:
import socket, ssl

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
ssl_context = ssl.SSLContext(ssl.PROTOCOL_TLS_SERVER)
ssl_context.load_cert_chain(certfile="server.crt", keyfile="server.key")

server_socket.bind(('localhost', 8443))
server_socket.listen(5)
print("SSL Server running...")

with ssl_context.wrap_socket(server_socket, server_side=True) as ssock:
    conn, addr = ssock.accept()
    data = conn.recv(1024).decode()
    print("Received:", data)
    conn.sendall(b"Hello from SSL Server!")
    conn.close()

Client Code

In [ ]:
import socket, ssl

hostname = 'localhost'

# Create an SSL context that ignores certificate verification (for demo)
context = ssl._create_unverified_context()

with socket.create_connection((hostname, 8443)) as sock:
    with context.wrap_socket(sock, server_hostname=hostname) as ssock:
        print("✅ SSL connection established.")
        ssock.sendall(b"Hello from SSL Client!")
        data = ssock.recv(1024).decode()
        print("Received:", data)

generate_cert.py

In [ ]:
from cryptography import x509
from cryptography.x509.oid import NameOID
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import rsa
from datetime import datetime, timedelta

# Generate private key
key = rsa.generate_private_key(public_exponent=65537, key_size=2048)

# Write private key to file
with open("server.key", "wb") as f:
    f.write(key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.TraditionalOpenSSL,
        encryption_algorithm=serialization.NoEncryption()
    ))

# Build self-signed certificate
subject = issuer = x509.Name([
    x509.NameAttribute(NameOID.COUNTRY_NAME, "IN"),
    x509.NameAttribute(NameOID.STATE_OR_PROVINCE_NAME, "Telangana"),
    x509.NameAttribute(NameOID.LOCALITY_NAME, "Hyderabad"),
    x509.NameAttribute(NameOID.ORGANIZATION_NAME, "SSL Demo"),
    x509.NameAttribute(NameOID.COMMON_NAME, "localhost"),
])

cert = (
    x509.CertificateBuilder()
    .subject_name(subject)
    .issuer_name(issuer)
    .public_key(key.public_key())
    .serial_number(x509.random_serial_number())
    .not_valid_before(datetime.utcnow())
    .not_valid_after(datetime.utcnow() + timedelta(days=365))
    .sign(key, hashes.SHA256())
)

# Write certificate to file
with open("server.crt", "wb") as f:
    f.write(cert.public_bytes(serialization.Encoding.PEM))

print("✅ Generated server.crt and server.key successfully!")